# Download Dataset
OntoNotes-5.0

In [8]:
pip install nltk

     |████████████████████████████████| 1.5 MB 1.7 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import nltk
nltk.download()
from nltk.corpus import brown

In [1]:
!git clone https://github.com/yuchenlin/OntoNotes-5.0-NER-BIO.git

Cloning into 'OntoNotes-5.0-NER-BIO'...
remote: Enumerating objects: 27517, done.
remote: Total 27517 (delta 0), reused 0 (delta 0), pack-reused 27517
Receiving objects: 100% (27517/27517), 101.80 MiB | 7.37 MiB/s, done.
Resolving deltas: 100% (19834/19834), done.
Updating files: 100% (26670/26670), done. files:  20% (5334/26670)


In [2]:
!git clone https://github.com/uclanlp/gn_glove.git

Cloning into 'gn_glove'...
remote: Enumerating objects: 199, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 199 (delta 2), reused 0 (delta 0), pack-reused 193
Receiving objects: 100% (199/199), 67.78 KiB | 1.17 MiB/s, done.
Resolving deltas: 100% (88/88), done.


In [3]:
!for path in $(find ./OntoNotes-5.0-NER-BIO/conll-formatted-ontonotes-5.0/data/ -name "*.gold_conll"); do sed 's/  */,/g; s/"/""/g' ${path} > ${path}.csv;done

# Load pre_trained model: BERT

In [4]:
pip install transformers

     |████████████████████████████████| 2.5 MB 2.9 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 7.4 MB/s  eta 0:00:01
     |████████████████████████████████| 2.2 MB 11.9 MB/s eta 0:00:01
     |████████████████████████████████| 249 kB 8.8 MB/s eta 0:00:01
     |████████████████████████████████| 285 kB 6.1 MB/s eta 0:00:01
     |████████████████████████████████| 895 kB 10.7 MB/s eta 0:00:01
     |████████████████████████████████| 178 kB 9.5 MB/s eta 0:00:01
     |████████████████████████████████| 145 kB 5.9 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 2.6 MB/s eta 0:00:01
     |████████████████████████████████| 138 kB 11.0 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'nltk'

In [6]:
# For DistilBERT:a version of BERT that is smaller, but much faster and requiring a lot less memory.
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/442 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Quantify bias 

In [5]:
import torch
import pandas as pd
import numpy as np
from pathlib import Path
from typing import *
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
!git clone https://github.com/allenai/allennlp.git

Cloning into 'allennlp'...
remote: Enumerating objects: 37918, done.
remote: Counting objects: 100% (744/744), done.
remote: Compressing objects: 100% (365/365), done.
^CKReceiving objects:  98% (37173/37918), 56.37 MiB | 3.72 MiB/sReceiving objects:  71% (27039/37918), 30.51 MiB | 3.27 MiB/s


In [6]:
def get_logits(sentence: str) -> np.ndarray:
    return model(processor.to_bert_model_input(sentence))[0, :, :].cpu().detach().numpy()

In [7]:
def softmax(arr, axis=1):
    e = np.exp(arr)
    return e / e.sum(axis=axis, keepdims=True)

In [8]:
def get_mask_fill_logits(sentence, words: Iterable[str],
                         use_last_mask=False, apply_softmax=False) -> Dict[str, float]:
    mask_i = processor.get_index(sentence, "[MASK]", last=use_last_mask)
    logits = defaultdict(list)
    out_logits = get_logits(sentence)
    if apply_softmax: 
        out_logits = softmax(out_logits)
    return {w: out_logits[mask_i, processor.token_to_index(w)] for w in words}

In [3]:
gender_words=['he','she']
mw, fw = gender_words
print(mw,fw)

he she


In [1]:
def bias_score(sentence, gender_words, target_words):# should be str 
    """
    Input a sentence of the form "GGG is XXX"
    XXX is a placeholder for the target word
    GGG is a placeholder for the gendered words (the subject)
    We will predict the bias when filling in the gendered words and 
    filling in the target word.
    """
    # probability of filling [MASK] with "he" vs. "she" when target is "programmer"
    mwords, fwords = gender_words
    subject_fill_logits = get_mask_fill_logits(
        sentence.replace("XXX", i for i in target_words).replace("GGG", "[MASK]"), # the sentence replaced by [mask]
        gender_words, use_last_mask=not gender_comes_first,
    )
    subject_fill_bias = subject_fill_logits[mwords] - subject_fill_logits[fwords]
    # male words are simply more likely than female words
    # correct for this by masking the target word and measuring the prior probabilities
    subject_fill_prior_logits = get_mask_fill_logits(
        sentence.replace("XXX", "[MASK]").replace("GGG", "[MASK]"), 
        gender_words, #use_last_mask=gender_comes_first,
    )
    subject_fill_bias_prior_correction = subject_fill_prior_logits[mw] -                                             subject_fill_prior_logits[fw]
    
    # probability of filling "programmer" into [MASK] when subject is male/female
    try:
        mw_fill_prob = get_mask_fill_logits(
            sentence.replace("GGG", mw).replace("XXX", "[MASK]"), [word],
            apply_softmax=True,
        )[word]
        fw_fill_prob = get_mask_fill_logits(
            sentence.replace("GGG", fw).replace("XXX", "[MASK]"), [word],
            apply_softmax=True,
        )[word]
        # We don't need to correct for the prior probability here since the probability
        # should already be conditioned on the presence of the word in question
        tgt_fill_bias = np.log(mw_fill_prob / fw_fill_prob)
    except:
        tgt_fill_bias = np.nan # TODO: handle multi word case
    return {"gender_fill_bias": subject_fill_bias,
            "gender_fill_prior_correction": subject_fill_bias_prior_correction,
            "gender_fill_bias_prior_corrected": subject_fill_bias - subject_fill_bias_prior_correction,
            "target_fill_bias": tgt_fill_bias, 
           }


NameError: name 'Iterable' is not defined